In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report, f1_score, plot_roc_curve

In [2]:
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
import matplotlib.patches as patches
from aif360.algorithms.preprocessing import Reweighing
#from packages import *
#from ml_fairness import *
import matplotlib.pyplot as plt
import seaborn as sns



from IPython.display import Markdown, display

In [3]:
adult = pd.read_csv('../../Data/adult.csv')
adult.sample()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
5898,35,Private,28160,Bachelors,13,Married-spouse-absent,Exec-managerial,Unmarried,White,Female,0,0,40,United-States,<=50K


In [4]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


*In this info detail, indicate that there is no missing value at all. But if you see the whole data carefully, you will find **missing value with '?'**.*

# PreProcessing

*Preprocessing scheme:*
* Encode all columns
* Drop education because it's already encoded on education.num
* Drop fnlwgt because it's unique

*Handling Missing Value In Pipeline*

In [5]:
binary_encoder_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'NC', missing_values = '?')),
    ('binary', ce.BinaryEncoder())
])

transformer = ColumnTransformer([
    ('one hot', OneHotEncoder(drop = 'first'), ['relationship', 'race', 'sex']),
    ('binary', binary_encoder_pipe, ['workclass', 'marital.status', 'occupation', 'native.country'])],
    remainder = 'passthrough')

*Splitting Data*

In [6]:
adult['income'].value_counts()

<=50K    24720
>50K      7841
Name: income, dtype: int64

Income is the target data and **indicated with imbalance data**. I define **income with 1 if income is >50K and 0 if income is <50K**.

In [7]:
X = adult.drop(['fnlwgt', 'education', 'income'], axis = 1)
y = np.where(adult['income'] == '>50K', 1, 0)

In [8]:
X.shape

(32561, 12)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y,
                                                    test_size = 0.3, random_state = 1212)

I use 0.3 as default score for test_size and X.shape for random_state so the data will be devided equally.

# Define Model

I use 3 Boosting Algorithms Models:
* Ada Boost Classifier
* Gradient Boosting Classifier
* XGB Classifier

In [10]:
adaboost = AdaBoostClassifier(DecisionTreeClassifier(), random_state = 1212)
pipe_ada = Pipeline([
    ('transformer', transformer),
    ('adaboost', adaboost)])

gradboost = GradientBoostingClassifier(random_state = 1212)
pipe_grad = Pipeline([
    ('transformer', transformer),
    ('gradboost', gradboost)])

XGBOOST = XGBClassifier(random_state = 1212)
pipe_XGB = Pipeline([
    ('transformer', transformer),
    ('XGBOOST', XGBOOST)])

# Cross Validation

*Model Evaluation*

In [12]:
def model_evaluation(model, metric):
    skfold = StratifiedKFold(n_splits = 5)
    model_cv = cross_val_score(model, X_train, y_train, cv = skfold, scoring = metric)
    return model_cv

pipe_ada_cv = model_evaluation(pipe_ada, 'f1')
pipe_grad_cv = model_evaluation(pipe_grad, 'f1')
pipe_XGB_cv = model_evaluation(pipe_XGB, 'f1')

*Fitting Data*

In [ ]:
for model in [pipe_ada, pipe_grad, pipe_XGB]:
    model.fit(X_train, y_train)

*Summary*

In [ ]:
score_mean = [pipe_ada_cv.mean(), pipe_grad_cv.mean(), pipe_XGB_cv.mean()]
score_std = [pipe_ada_cv.std(), pipe_grad_cv.std(), pipe_XGB_cv.std()]
score_f1 = [f1_score(y_test, pipe_ada.predict(X_test)),
            f1_score(y_test, pipe_grad.predict(X_test)), 
            f1_score(y_test, pipe_XGB.predict(X_test))]
method_name = ['Ada Boost Classifier', 'Gradient Boost Classifier ',
              'XGB Classifier']
summary = pd.DataFrame({'method': method_name, 'mean score': score_mean,
                        'std score': score_std, 'f1 score': score_f1})
summary

From these scores, **XGB Classifier is the best one** with the highest f1 score and mean score, also the lowest std score. Let's cross-check with the important features, see if the model is correct.

In [ ]:
plot_roc_curve(pipe_XGB, X_test, y_test)

# Importance Features

In [ ]:
features = list(pipe_ada[0].transformers_[0][1].get_feature_names()) + pipe_ada[0].transformers_[1][1][1].get_feature_names() + ['age', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

In [ ]:
imptab_ada = pd.DataFrame(pipe_ada[1].feature_importances_, columns = ['imp'], index = features)
imptab_ada.sort_values('imp').plot(kind = 'barh', figsize = (15,8))
plt.title('Importance Table For Ada Boost Classifier Model')
plt.show()

In [ ]:
imptab_grad = pd.DataFrame(pipe_grad[1].feature_importances_, columns = ['imp'], index = features)
imptab_grad.sort_values('imp').plot(kind = 'barh', figsize = (15,8))
plt.title('Importance Table For Gradient Boost Classifier Model')
plt.show()

In [ ]:
imptab_XGB = pd.DataFrame(pipe_XGB[1].feature_importances_, columns = ['imp'], index = features)
imptab_XGB.sort_values('imp').plot(kind = 'barh', figsize = (15,8))
plt.title('Importance Table For XGB Classifier Model')
plt.show()

From Importance Features Table, the **XGB Classifier can boost almost all the features**. It's has a consistency with the cross validation result. Now, see if the HyperParameter Tuning process can boost until getting the maximum score.

# HyperParameter Tuning

In [ ]:
XGBOOST = XGBClassifier(random_state = 1212)
estimator = Pipeline([('transformer', transformer), ('XGBOOST', XGBOOST)])

hyperparam_space = {
    'XGBOOST__learning_rate': [0.1, 0.05, 0.01, 0.005],
    'XGBOOST__n_estimators': [50, 100, 150, 200],
    'XGBOOST__max_depth': [3, 5, 7, 9]
}

random = RandomizedSearchCV(
                estimator,
                param_distributions = hyperparam_space,
                cv = StratifiedKFold(n_splits = 5),
                scoring = 'f1',
                n_iter = 10,
                n_jobs = -1)

random.fit(X_train, y_train)

In [19]:
print('best score', random.best_score_)
print('best param', random.best_params_)

best score 0.7017567140030995
best param {'XGBOOST__n_estimators': 200, 'XGBOOST__max_depth': 9, 'XGBOOST__learning_rate': 0.05}


After HyperParameter Tuning, the best score is 0.6996, which getting lower. N estimator is 150, Max depth is 5, and Learning rate is 0.1. Let's compare the result.

# Before VS After Tuning Comparison

In [20]:
estimator.fit(X_train, y_train)
y_pred_estimator = estimator.predict(X_test)
print(classification_report(y_test, y_pred_estimator))

[06:01:38] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      7417
           1       0.77      0.66      0.71      2352

    accuracy                           0.87      9769
   macro avg       0.83      0.80      0.82      9769
weighted avg       0.87      0.87      0.87      9769



In [21]:
random.best_estimator_.fit(X_train, y_train)
y_pred_random = random.best_estimator_.predict(X_test)
print(classification_report(y_test, y_pred_random))

[06:01:40] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      7417
           1       0.78      0.64      0.70      2352

    accuracy                           0.87      9769
   macro avg       0.84      0.79      0.81      9769
weighted avg       0.86      0.87      0.87      9769



In [22]:
score_list = [f1_score(y_test, y_pred_estimator), f1_score(y_test, y_pred_random)]
method_name = ['XGB Classifier Before Tuning', 'XGB Classifier After Tuning']
best_summary = pd.DataFrame({
    'method': method_name,
    'f1 score': score_list
})
best_summary

,method,f1 score
0,XGB Classifier Before Tuning,0.713175
1,XGB Classifier After Tuning,0.703169


After all, HyperParameter Tuning doesn't work good in this data. So if I have to choose, I pick the **XGB Classifier score Before Tuning, which is 0.71**. I know the number isn't good enough either because the data is imbalance and I don't process any resampling on it.

## Fairness

In [7]:
# This DataFrame is created to stock differents models and fair metrics that we produce in this notebook
algo_metrics = pd.DataFrame(columns=['model', 'fair_metrics', 'prediction', 'probs'])

def add_to_df_algo_metrics(algo_metrics, model, fair_metrics, preds, probs, name):
    return algo_metrics.append(pd.DataFrame(data=[[model, fair_metrics, preds, probs]], columns=['model', 'fair_metrics', 'prediction', 'probs'], index=[name]))

In [8]:
def fair_metrics(dataset, pred, pred_is_dataset=False):
    if pred_is_dataset:
        dataset_pred = pred
    else:
        dataset_pred = dataset.copy()
        dataset_pred.labels = pred
    
    cols = ['statistical_parity_difference', 'equal_opportunity_difference', 'average_abs_odds_difference',  'disparate_impact', 'theil_index']
    obj_fairness = [[0,0,0,1,0]]
    
    fair_metrics = pd.DataFrame(data=obj_fairness, index=['objective'], columns=cols)
    
    for attr in dataset_pred.protected_attribute_names:
        idx = dataset_pred.protected_attribute_names.index(attr)
        privileged_groups =  [{attr:dataset_pred.privileged_protected_attributes[idx][0]}] 
        unprivileged_groups = [{attr:dataset_pred.unprivileged_protected_attributes[idx][0]}] 
        
        classified_metric = ClassificationMetric(dataset, 
                                                     dataset_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)

        metric_pred = BinaryLabelDatasetMetric(dataset_pred,
                                                     unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)

        acc = classified_metric.accuracy()

        row = pd.DataFrame([[metric_pred.mean_difference(),
                                classified_metric.equal_opportunity_difference(),
                                classified_metric.average_abs_odds_difference(),
                                metric_pred.disparate_impact(),
                                classified_metric.theil_index()]],
                           columns  = cols,
                           index = [attr]
                          )
        fair_metrics = fair_metrics.append(row)    
    
    fair_metrics = fair_metrics.replace([-np.inf, np.inf], 2)
        
    return fair_metrics

def plot_fair_metrics(fair_metrics):
    fig, ax = plt.subplots(figsize=(20,4), ncols=5, nrows=1)

    plt.subplots_adjust(
        left    =  0.125, 
        bottom  =  0.1, 
        right   =  0.9, 
        top     =  0.9, 
        wspace  =  .5, 
        hspace  =  1.1
    )

    y_title_margin = 1.2

    plt.suptitle("Fairness metrics", y = 1.09, fontsize=20)
    sns.set(style="dark")

    cols = fair_metrics.columns.values
    obj = fair_metrics.loc['objective']
    size_rect = [0.2,0.2,0.2,0.4,0.25]
    rect = [-0.1,-0.1,-0.1,0.8,0]
    bottom = [-1,-1,-1,0,0]
    top = [1,1,1,2,1]
    bound = [[-0.1,0.1],[-0.1,0.1],[-0.1,0.1],[0.8,1.2],[0,0.25]]

    display(Markdown("### Check bias metrics :"))
    display(Markdown("A model can be considered bias if just one of these five metrics show that this model is biased."))
    for attr in fair_metrics.index[1:len(fair_metrics)].values:
        display(Markdown("#### For the %s attribute :"%attr))
        check = [bound[i][0] < fair_metrics.loc[attr][i] < bound[i][1] for i in range(0,5)]
        display(Markdown("With default thresholds, bias against unprivileged group detected in **%d** out of 5 metrics"%(5 - sum(check))))

    for i in range(0,5):
        plt.subplot(1, 5, i+1)
        ax = sns.barplot(x=fair_metrics.index[1:len(fair_metrics)], y=fair_metrics.iloc[1:len(fair_metrics)][cols[i]])
        
        for j in range(0,len(fair_metrics)-1):
            a, val = ax.patches[j], fair_metrics.iloc[j+1][cols[i]]
            marg = -0.2 if val < 0 else 0.1
            ax.text(a.get_x()+a.get_width()/5, a.get_y()+a.get_height()+marg, round(val, 3), fontsize=15,color='black')

        plt.ylim(bottom[i], top[i])
        plt.setp(ax.patches, linewidth=0)
        ax.add_patch(patches.Rectangle((-5,rect[i]), 10, size_rect[i], alpha=0.3, facecolor="green", linewidth=1, linestyle='solid'))
        plt.axhline(obj[i], color='black', alpha=0.3)
        plt.title(cols[i])
        ax.set_ylabel('')    
        ax.set_xlabel('')

In [9]:
def get_fair_metrics_and_plot(data, model, plot=False, model_aif=False):
    pred = model.predict(data).labels if model_aif else model.predict(data.features)
    # fair_metrics function available in the metrics.py file
    fair = fair_metrics(data, pred)

    if plot:
        # plot_fair_metrics function available in the visualisations.py file
        # The visualisation of this function is inspired by the dashboard on the demo of IBM aif360 
        plot_fair_metrics(fair)
        display(fair)
    
    return fair

In [13]:
##train['Sex'] = train['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
adult_df = adult.drop(['fnlwgt', 'education'], axis = 1)
adult_df["income"]=adult_df["income"].map({"<=50K":0,">50K":1})
adult_df["sex"] = adult_df["sex"].map({"Male":1,"Female":0})
adult_df["workclass"] = adult_df["workclass"].replace("?",np.nan)
adult_df["occupation"] = adult_df["occupation"].replace("?",np.nan)
adult_df["native.country"] = adult_df["native.country"].replace("?",np.nan)
adult_df.head()
#features = ["Pclass", "Sex", "SibSp", "Parch", "Survived"]
#X = pd.get_dummies(train_data[features])

,age,workclass,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,9,Widowed,NaN,Not-in-family,White,0,0,4356,40,United-States,0
1,82,Private,9,Widowed,Exec-managerial,Not-in-family,White,0,0,4356,18,United-States,0
2,66,NaN,10,Widowed,NaN,Unmarried,Black,0,0,4356,40,United-States,0
3,54,Private,4,Divorced,Machine-op-inspct,Unmarried,White,0,0,3900,40,United-States,0
4,41,Private,10,Separated,Prof-specialty,Own-child,White,0,0,3900,40,United-States,0


In [14]:
adult_df["workclass"] = adult_df["workclass"].fillna(adult_df["workclass"].mode()[0])
adult_df["occupation"] = adult_df["occupation"].fillna(adult_df["occupation"].mode()[0])
adult_df["native.country"] = adult_df["native.country"].fillna(adult_df["native.country"].mode()[0])

In [15]:
from sklearn import preprocessing

categorical = ['workclass', 'marital.status', 'occupation', 'relationship','race','native.country',]
for feature in categorical:
        print(feature)
        le = preprocessing.LabelEncoder()
        adult_df[feature] = le.fit_transform(adult_df[feature])
        #X_test[feature] = le.transform(X_test[feature])

workclass
marital.status
occupation
relationship
race
native.country


In [16]:
adult_df.head()

,age,workclass,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,3,9,6,9,1,4,0,0,4356,40,38,0
1,82,3,9,6,3,1,4,0,0,4356,18,38,0
2,66,3,10,6,9,4,2,0,0,4356,40,38,0
3,54,3,4,0,6,4,4,0,0,3900,40,38,0
4,41,3,10,5,9,3,4,0,0,3900,40,38,0


In [17]:
from scipy.stats import pearsonr

pearsonr(adult_df['income'], adult_df['sex'])




(0.21598015058404135, 0.0)

In [20]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
dataset_orig = StandardDataset(adult_df,
                                  label_name='income',
                                  protected_attribute_names=['sex'],
                                  favorable_classes=[1],
                                  privileged_classes=[[1]])

In [21]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.196276


In [10]:
data_orig_train = pd.read_pickle('../../Results/AdaBoost/2-boosting-algorithms-model-for-adult-census-income_Train.pkl')
data_orig_test = pd.read_pickle('../../Results/AdaBoost/2-boosting-algorithms-model-for-adult-census-income_Test.pkl')

In [11]:
import ipynbname
nb_fname = ipynbname.name()
nb_path = ipynbname.path()

from sklearn.ensemble import AdaBoostClassifier
import pickle

#data_orig_train, data_orig_test = dataset_orig.split([0.7], shuffle=True)
X_train = data_orig_train.features
y_train = data_orig_train.labels.ravel()

X_test = data_orig_test.features
y_test = data_orig_test.labels.ravel()
num_estimators = 100

model = AdaBoostClassifier(DecisionTreeClassifier(), random_state = 1212, n_estimators= 1)

mdl = model.fit(X_train, y_train)
#with open('../../Results/AdaBoost/' + nb_fname + '.pkl', 'wb') as f:
#        pickle.dump(mdl, f)

#with open('../../Results/AdaBoost/' + nb_fname + '_Train' + '.pkl', 'wb') as f:
#    pickle.dump(data_orig_train, f) 
    
#with open('../../Results/AdaBoost/' + nb_fname + '_Test' + '.pkl', 'wb') as f:
#    pickle.dump(data_orig_test, f) 

In [26]:
from csv import writer
from sklearn.metrics import accuracy_score, f1_score

final_metrics = []
accuracy = []
f1= []

for i in range(1,num_estimators+1):
    
    model = AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy', splitter = 'random', max_depth = 1), random_state = 1212, n_estimators= i)
    
    mdl = model.fit(X_train, y_train)
    yy = mdl.predict(X_test)
    accuracy.append(accuracy_score(y_test, yy))
    f1.append(f1_score(y_test, yy))
    fair = get_fair_metrics_and_plot(data_orig_test, mdl)                           
    fair_list = fair.iloc[1].tolist()
    #fair_list.insert(0, i)
    final_metrics.append(fair_list)


In [27]:
import numpy as np
final_result = pd.DataFrame(final_metrics)
print(final_result)
final_result[3] = np.log(final_result[3])
final_result = final_result.transpose()
acc_f1 = pd.DataFrame(accuracy)
acc_f1['f1'] = f1
acc_f1 = pd.DataFrame(acc_f1).transpose()
acc = acc_f1.rename(index={0: 'accuracy', 1: 'f1'})
final_result = final_result.rename(index={0: 'statistical_parity_difference', 1: 'equal_opportunity_difference', 2: 'average_abs_odds_difference', 3: 'disparate_impact', 4: 'theil_index'})
final_result = pd.concat([acc,final_result])
final_result.columns = ['T' + str(col) for col in final_result.columns]
final_result.insert(0, "classifier", final_result['T' + str(num_estimators - 1)])   ##Add final metrics add the beginning of the df
#final_result.to_csv('../../Results/AdaBoost/' + nb_fname + '.csv')
final_result

           0         1         2         3         4
0   0.000000  0.000000  0.000000       NaN  0.271671
1   0.000000  0.000000  0.000000       NaN  0.271671
2  -0.435397 -0.718274  0.515449  0.000000  0.140487
3  -0.021355 -0.012690  0.018903  0.000000  0.274012
4  -0.456752 -0.730964  0.534352  0.000000  0.139671
..       ...       ...       ...       ...       ...
95 -0.182016 -0.143454  0.111036  0.269581  0.126545
96 -0.182169 -0.145264  0.112051  0.268516  0.126926
97 -0.183398 -0.147294  0.113555  0.268094  0.126696
98 -0.183859 -0.147294  0.113837  0.266706  0.126696
99 -0.184166 -0.145485  0.113153  0.267274  0.126415

[100 rows x 5 columns]


,classifier,T0,T1,T2,T3,T4,T5,T6,T7,T8,...,T90,T91,T92,T93,T94,T95,T96,T97,T98,T99
accuracy,0.845941,0.762105,0.762105,0.761695,0.752994,0.752585,0.808169,0.809499,0.814208,0.812673,...,0.846351,0.845839,0.845737,0.845225,0.845737,0.846555,0.846146,0.845839,0.845839,0.845941
f1,0.639866,0.000000,0.000000,0.548662,0.020300,0.543704,0.443917,0.449897,0.503148,0.482173,...,0.641509,0.640057,0.639560,0.639485,0.639215,0.640096,0.638962,0.639195,0.639195,0.639866
statistical_parity_difference,-0.184166,0.000000,0.000000,-0.435397,-0.021355,-0.456752,-0.160701,-0.161317,-0.202337,-0.173927,...,-0.184475,-0.184474,-0.184934,-0.185090,-0.183399,-0.182016,-0.182169,-0.183398,-0.183859,-0.184166
equal_opportunity_difference,-0.145485,0.000000,0.000000,-0.718274,-0.012690,-0.730964,-0.379695,-0.376297,-0.453168,-0.365838,...,-0.145198,-0.146500,-0.152149,-0.147515,-0.144469,-0.143454,-0.145264,-0.147294,-0.147294,-0.145485
average_abs_odds_difference,0.113153,0.000000,0.000000,0.515449,0.018903,0.534352,0.222674,0.220975,0.271748,0.220662,...,0.112947,0.113709,0.116423,0.114533,0.112363,0.111036,0.112051,0.113555,0.113837,0.113153
disparate_impact,-1.319479,NaN,NaN,-inf,-inf,-inf,-inf,-5.172173,-5.111415,-3.127099,...,-1.314049,-1.317365,-1.325888,-1.313182,-1.313088,-1.310885,-1.314845,-1.316418,-1.321608,-1.319479
theil_index,0.126415,0.271671,0.271671,0.140487,0.274012,0.139671,0.188861,0.187256,0.172201,0.178419,...,0.125786,0.126283,0.126481,0.126316,0.126646,0.126545,0.126926,0.126696,0.126696,0.126415
